In [2]:
import os
import torch
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
from pixelmed_calc.medical_imaging.modelstore import get_model_from_name
#可改参数
model='resnet50_3D'
num_classes=2
pretrained=True
in_channels=1
batch_size=4
learning_rate=1e-4
epochs=10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#video = torch.randn(4, 1, 16, 128, 128).to("cuda") # (batch, channels, frames, height, width)

model=get_model_from_name['ResViT'](
    image_size = 128,          # image size
    frames = 16,               # number of frames
    image_patch_size = 16,     # image patch size
    frame_patch_size = 2,      # frame patch size
    num_classes = 2,
    channels=1,
    dim = 1024,
    depth = 6,
    heads = 8,
    mlp_dim = 2048,
    dropout = 0.1,
    emb_dropout = 0.1
).to(device)

#out=model(video)

In [ ]:
from pixelmed_calc.medical_imaging.DLtools.process import load_data_from_csv,get_file_and_labels,get_transforms,train_and_validate
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
import torch
import torch.nn as nn
from monai.data import ImageDataset
import os
import pandas as pd
from torch.utils.data import DataLoader

# CSV文件路径
csv_path = r"D:\pixelmedAI\function_pm\pm_data\liangxing\group.csv"#'E:\127lungnode\lungnode\group.csv'
train_base_path = r"D:\pixelmedAI\function_pm\pm_data\liangxing\images"#'E:\127lungnode\lungnode\crop3d'
val_base_path = r"D:\pixelmedAI\function_pm\pm_data\liangxing\images"#'E:\127lungnode\lungnode\crop3d'
# CSV文件路径
tfs=get_transforms(resize=[16,128,128])
# 读取所有数据
file_names, labels, groups = load_data_from_csv(csv_path)
# 获取训练数据和标签（可以设置 base_path 为 None 或实际路径）
file_train, train_labels = get_file_and_labels(file_names, labels, groups, 'train', train_base_path)
# 获取验证数据和标签（可以设置 base_path 为 None 或实际路径）
file_val, val_labels = get_file_and_labels(file_names, labels, groups, 'val', val_base_path)

# 假设你已经定义了 ImageDataset 和 train_transforms, val_transforms
train_ds = ImageDataset(image_files=file_train, labels=train_labels, transform=tfs)
train_loader = DataLoader(train_ds, batch_size=4, shuffle=False, num_workers=0, pin_memory=torch.cuda.is_available())

val_ds = ImageDataset(image_files=file_val, labels=val_labels, transform=tfs)
val_loader = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=0, pin_memory=torch.cuda.is_available())

训练模型

In [ ]:
train_and_validate(model, train_loader, val_loader, learning_rate=1e-4, epochs=200)